In [1]:
from jeroHelper.setupUtils import append_deepfinder_path, get_tomo_indices, PARENT_PATH
append_deepfinder_path()

from jeroHelper.trainHelper import make_trainer
from jeroHelper.coordGen import OLMaker

from deepfinder.utils.common import read_array
import deepfinder.utils.objl as ol


import numpy as np
import matplotlib.pyplot as plt
import random
import mrcfile
import pandas as pd

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

# Reconstruction model

In [2]:
train_tomos = ['tomo02', 'tomo03', 'tomo04', 'tomo17']
concat_train_ids = sorted([s.replace('tomo', '') for s in train_tomos])
concat_train_ids = '-'.join(concat_train_ids)

val_tomos = ['tomo32', 'tomo10']
concat_val_ids = sorted([s.replace('tomo', '') for s in val_tomos])
concat_val_ids = '-'.join(concat_val_ids)

test_tomos = ['tomo38']

objectList_maker = OLMaker(train_tomos, val_tomos, test_tomos, 12, 15, 20, None, None)

objectList_maker.make_xml_objlists(overwrite=False)
rsample_train, rsample_val = ol.read_xml(objectList_maker.train_objlist_path), ol.read_xml(objectList_maker.val_objlist_path)

Pairs of tomo IDs to indices:
[('tomo02_patch000', 0), ('tomo02_patch001', 1), ('tomo02_patch002', 2), ('tomo02_patch003', 3), ('tomo02_patch004', 4), ('tomo02_patch005', 5), ('tomo02_patch006', 6), ('tomo02_patch020', 7), ('tomo02_patch021', 8), ('tomo02_patch030', 9), ('tomo03_patch000', 10), ('tomo03_patch002', 11), ('tomo03_patch004', 12), ('tomo03_patch006', 13), ('tomo03_patch008', 14), ('tomo03_patch010', 15), ('tomo03_patch020', 16), ('tomo03_patch021', 17), ('tomo03_patch022', 18), ('tomo03_patch030', 19), ('tomo03_patch031', 20), ('tomo03_patch032', 21), ('tomo04_patch000', 22), ('tomo04_patch002', 23), ('tomo04_patch004', 24), ('tomo04_patch006', 25), ('tomo04_patch008', 26), ('tomo04_patch009', 27), ('tomo04_patch020', 28), ('tomo04_patch021', 29), ('tomo04_patch030', 30), ('tomo10_patch001', 31), ('tomo10_patch002', 32), ('tomo10_patch003', 33), ('tomo10_patch009', 34), ('tomo10_patch010', 35), ('tomo10_patch011', 36), ('tomo10_patch020', 37), ('tomo10_patch030', 38), ('to

In [3]:
objectList_maker.path_data, objectList_maker.path_target

(['/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch000_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch001_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch002_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch003_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch004_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch005_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch006_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch020_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thesis/data/nnUnet/Task143_cryoET7/imagesTr/tomo02_patch021_0000.nii.gz',
  '/home/haicu/jeronimo.carvajal/Thes

In [3]:
tb_logdir = './logs_IsenseePatches_main/2.10_reconstructionModel/'

trainer = make_trainer(dim_in=56, batch_size=32, lr=5e-5, epochs=100, tb_logdir=tb_logdir, model_name='2.10_reconstructionModel',
                       reconstruction_trainer=True, pretrained_model=None)
trainer.launch(objectList_maker.path_data, objectList_maker.path_target, rsample_train, rsample_val)

Selecting trainer for reconstruction task... Ignoring pretrained_model value.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Set SLURM handle signals.

  | Name    | Type       | Params
---------------------------------------
0 | loss_fn | MSELoss    | 0     
1 | layer1  | Sequential | 28.6 K
2 | layer2  | Sequential | 103 K 
3 | layer3  | Sequential | 558 K 
4 | layer4  | Sequential | 288 K 
5 | layer5  | Sequential | 96.9 K
---------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.303     Total estimated model params size (MB)


"loss_fn":      MSELoss()
"lr":           5e-05
"weight_decay": 0.0
Saving model at:  /home/haicu/jeronimo.carvajal/Thesis/models/2.10_reconstructionModel_ep100_in56_lr0.000050_v2.model

Writing to modelSummary...
Opening ModelSummary file...
ModelSummary file exists, appending current model data...


# Pretrained model with reconstruction task model

In [4]:
train_tomos = ['tomo02']
concat_train_ids = sorted([s.replace('tomo', '') for s in train_tomos])
concat_train_ids = '-'.join(concat_train_ids)

val_tomos = ['tomo32', 'tomo10']
concat_val_ids = sorted([s.replace('tomo', '') for s in val_tomos])
concat_val_ids = '-'.join(concat_val_ids)

test_tomos = ['tomo38']

objectList_maker = OLMaker(train_tomos, val_tomos, test_tomos, 60, 15, 20, None, None)

objectList_maker.make_xml_objlists(overwrite=False)
rsample_train, rsample_val = ol.read_xml(objectList_maker.train_objlist_path), ol.read_xml(objectList_maker.val_objlist_path)

random.shuffle(rsample_train)

rsample_train = rsample_train[0:492]

Pairs of tomo IDs to indices:
[('tomo02_patch000', 0), ('tomo02_patch001', 1), ('tomo02_patch002', 2), ('tomo02_patch003', 3), ('tomo02_patch004', 4), ('tomo02_patch005', 5), ('tomo02_patch006', 6), ('tomo02_patch020', 7), ('tomo02_patch021', 8), ('tomo02_patch030', 9), ('tomo03_patch000', 10), ('tomo03_patch002', 11), ('tomo03_patch004', 12), ('tomo03_patch006', 13), ('tomo03_patch008', 14), ('tomo03_patch010', 15), ('tomo03_patch020', 16), ('tomo03_patch021', 17), ('tomo03_patch022', 18), ('tomo03_patch030', 19), ('tomo03_patch031', 20), ('tomo03_patch032', 21), ('tomo04_patch000', 22), ('tomo04_patch002', 23), ('tomo04_patch004', 24), ('tomo04_patch006', 25), ('tomo04_patch008', 26), ('tomo04_patch009', 27), ('tomo04_patch020', 28), ('tomo04_patch021', 29), ('tomo04_patch030', 30), ('tomo10_patch001', 31), ('tomo10_patch002', 32), ('tomo10_patch003', 33), ('tomo10_patch009', 34), ('tomo10_patch010', 35), ('tomo10_patch011', 36), ('tomo10_patch020', 37), ('tomo10_patch030', 38), ('to

In [ ]:
tb_logdir = './logs_IsenseePatches_main/2.10_preTrainedReconstructionTaskModel/'

trainer = make_trainer(dim_in=56, batch_size=32, lr=1e-4, epochs=700, tb_logdir=tb_logdir,
                       model_name='2.10_preTrainedReconstructionTaskModel',
                       reconstruction_trainer=False, 
                       pretrained_model=PARENT_PATH+'models/2.10_reconstructionModel_ep100_in56_lr0.000050_v2.model')

trainer.launch(objectList_maker.path_data, objectList_maker.path_target, rsample_train, rsample_val)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Set SLURM handle signals.

  | Name    | Type         | Params
-----------------------------------------
0 | loss_fn | Tversky_loss | 0     
1 | layer1  | Sequential   | 28.6 K
2 | layer2  | Sequential   | 103 K 
3 | layer3  | Sequential   | 558 K 
4 | layer4  | Sequential   | 288 K 
5 | layer5  | Sequential   | 96.9 K
-----------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.304     Total estimated model params size (MB)


"Ncl":           3
"loss_fn":       Tversky_loss()
"lr":            0.0001
"pretrain_type": reconstructionTask
"weight_decay":  0.0
